In [1]:
R.version.string

[1] "R version 4.1.1 (2021-08-10)"

In [17]:
library(data.table)
install.packages("text2vec")
install.packages("caTools")
install.packages("data.table")
install.packages("tidyverse")
install.packages("glmnet")
install.packages("inspectdf")
install.packages("magrittr")
install.packages("dplyr")
library(tidyverse)
library(text2vec)
library(caTools)
library(glmnet)
library(inspectdf)
library(magrittr)
library(dplyr)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)



In [18]:
df <- read.csv("/emails.csv")
str(df)

'data.frame':	5728 obs. of  2 variables:
 $ text: chr  "Subject: naturally irresistible your corporate identity  lt is really hard to recollect a company : the  market"| __truncated__ "Subject: the stock trading gunslinger  fanny is merrill but muzo not colza attainder and penultimate like esmar"| __truncated__ "Subject: unbelievable new homes made easy  im wanting to show you this  homeowner  you have been pre - approved"| __truncated__ "Subject: 4 color printing special  request additional information now ! click here  click here for a printable "| __truncated__ ...
 $ spam: int  1 1 1 1 1 1 1 1 1 1 ...


In [19]:
colnames(df)

[1] "text" "spam"

In [20]:
df$ID <- seq.int(nrow(df))
df <- df %>% select(ID, everything())

In [21]:
colnames(df)

[1] "ID"   "text" "spam"

In [22]:
df %>% inspect_na()

col_name,cnt,pcnt
<chr>,<int>,<dbl>
ID,0,0
text,0,0
spam,0,0


In [24]:
glimpse(df)

Rows: 5,728
Columns: 3
$ ID   <int> 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19…
$ text <chr> "Subject: naturally irresistible your corporate identity  lt is r…
$ spam <int> 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,…


In [25]:
df$ID <- df$ID %>% as.character()

In [26]:
glimpse(df)

Rows: 5,728
Columns: 3
$ ID   <chr> "1", "2", "3", "4", "5", "6", "7", "8", "9", "10", "11", "12", "1…
$ text <chr> "Subject: naturally irresistible your corporate identity  lt is r…
$ spam <int> 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,…


In [27]:
# Split data
set.seed(123)
split <- df$spam %>% sample.split(SplitRatio = 0.8)
train <- df %>% subset(split == T)
test <- df %>% subset(split == F)

In [28]:
#Tokenize
it_train <- train$text %>% 
  itoken(preprocessor = tolower, 
         tokenizer = word_tokenizer,
         ids = train$ID,
         progressbar = F) 

train_tokens <- train$text %>% tolower() %>% word_tokenizer()

it_train <- train_tokens %>% 
  itoken(ids = train$ID,
         progressbar = F)

vocab <- it_train %>% create_vocabulary()
vocab %>% 
  arrange(desc(term_count)) %>% 
  head(110) %>% 
  tail(10) 

vectorizer <- vocab %>% vocab_vectorizer()
dtm_train <- it_train %>% create_dtm(vectorizer)

dtm_train %>% dim()
identical(rownames(dtm_train), train$ID)

,term,term_count,doc_count
,<chr>,<int>,<int>
101,there,1350,935
102,out,1340,908
103,which,1334,809
104,company,1326,547
105,12,1323,762
106,shirley,1308,578
107,4,1299,778
108,edu,1295,454
109,no,1280,809


[1]  4582 33598

[1] TRUE

In [29]:
#---Modeling---
glmnet_classifier <- dtm_train %>% 
  cv.glmnet(y = train[['spam']],
            family = 'binomial', 
            type.measure = "auc",
            nfolds = 10,
            thresh = 0.001,# high value is less accurate, but has faster training
            maxit = 1000)# again lower number of iterations for faster training

glmnet_classifier$cvm %>% max() %>% round(3) %>% paste("-> Max AUC")


it_test <- test$text %>% tolower() %>% word_tokenizer()

it_test <- it_test %>% 
  itoken(ids = test$ID,
         progressbar = F)

dtm_test <- it_test %>% create_dtm(vectorizer)

preds <- predict(glmnet_classifier, dtm_test, type = 'response')[,1]
glmnet:::auc(test$spam, preds) %>% round(2)


# Prune some words ----
stop_words <- c("i", "you", "he", "she", "it", "we", "they",
                "me", "him", "her", "them",
                "my", "your", "yours", "his", "our", "ours",
                "myself", "yourself", "himself", "herself", "ourselves",
                "the", "a", "an", "and", "or", "on", "by", "so",
                "from", "about", "to", "for", "of", 
                "that", "this", "is", "are")

vocab <- it_train %>% create_vocabulary(stopwords = stop_words)


pruned_vocab <- vocab %>% 
  prune_vocabulary(term_count_min = 10, 
                   doc_proportion_max = 0.5,
                   doc_proportion_min = 0.001)

pruned_vocab %>% 
  arrange(desc(term_count)) %>% 
  head(10) 

vectorizer <- pruned_vocab %>% vocab_vectorizer()

dtm_train <- it_train %>% create_dtm(vectorizer)
dtm_train %>% dim()

glmnet_classifier <- dtm_train %>% 
  cv.glmnet(y = train[['spam']], 
            family = 'binomial',
            type.measure = "auc",
            nfolds = 4,
            thresh = 0.001,
            maxit = 1000)

glmnet_classifier$cvm %>% max() %>% round(3) %>% paste("-> Max AUC")

dtm_test <- it_test %>% create_dtm(vectorizer)
preds <- predict(glmnet_classifier, dtm_test, type = 'response')[,1]
glmnet:::auc(test$spam, preds) %>% round(2)

[1] "0.994 -> Max AUC"

[1] 0.99

,term,term_count,doc_count
,<chr>,<int>,<int>
1,enron,10701,2078
2,ect,9106,1294
3,vince,6646,2219
4,at,5373,2238
5,s,5121,1923
6,as,4913,1906
7,hou,4455,1212
8,com,4304,1556
9,if,4110,2181


[1] 4582 6643

[1] "0.995 -> Max AUC"

[1] 0.99